In [1]:
from google.colab import drive
drive.mount("/content/gdrive")
!ls "/content/gdrive/MyDrive/data"

Mounted at /content/gdrive
 face.csv	        faceLabel.csv
 faceData	        model4emotionClassfier
 faceData2D	        model4multiClassfier
 faceData2DWithNumber  'model4multiClassfier(exceptAge)'
 face_Dataset	        model4raceClassfier
 faceDataset	        model4sexClassfier
 faceDataset2	        model4sexClassfierWithoutValid
 faceDataWithNumber     rawdata
 faceDR		        rawdata.zip
 faceDS		        vgg_face_dag.pth


In [2]:
import matplotlib.pyplot as plt  # show image
import numpy as np
import pandas as pd
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


import os
import cv2
import seaborn

from sklearn.svm import SVC
from skimage.feature import hog
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
# from sklearn.metrics import GridSearchCV

In [3]:
dtype = np.uint8
shape = (128,128)

# file_path /content/gdrive/My Drive/data/rawdata/xxxx
pre_img_path="/content/gdrive/MyDrive/data/rawdata/"
pre_path="/content/gdrive/MyDrive/data/"

# Read the data from the binary file
# data = np.fromfile(file_path, dtype=dtype).reshape(shape)

In [4]:
!unzip "/content/gdrive/MyDrive/data/rawdata.zip" -d /content/sample_data/rawdataFolder

Archive:  /content/gdrive/MyDrive/data/rawdata.zip
   creating: /content/sample_data/rawdataFolder/rawdata/
  inflating: /content/sample_data/rawdataFolder/rawdata/1223  
  inflating: /content/sample_data/rawdataFolder/rawdata/1224  
  inflating: /content/sample_data/rawdataFolder/rawdata/1225  
  inflating: /content/sample_data/rawdataFolder/rawdata/1226  
  inflating: /content/sample_data/rawdataFolder/rawdata/1227  
  inflating: /content/sample_data/rawdataFolder/rawdata/1229  
  inflating: /content/sample_data/rawdataFolder/rawdata/1230  
  inflating: /content/sample_data/rawdataFolder/rawdata/1231  
  inflating: /content/sample_data/rawdataFolder/rawdata/1233  
  inflating: /content/sample_data/rawdataFolder/rawdata/1234  
  inflating: /content/sample_data/rawdataFolder/rawdata/1235  
  inflating: /content/sample_data/rawdataFolder/rawdata/1236  
  inflating: /content/sample_data/rawdataFolder/rawdata/1237  
  inflating: /content/sample_data/rawdataFolder/rawdata/1238  
  inflatin

In [5]:
pre_img_path="/content/sample_data/rawdataFolder/"

In [6]:
!ls pre_img_path

ls: cannot access 'pre_img_path': No such file or directory


In [7]:
def preprocessing(src):
    # gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)  # 将图像转换成灰度图
    # img = cv2.resize(gray, (256, 256))  # 尺寸调整g
    img = src/255.0    # 数据归一化
    return img

In [8]:
def extract_hog_features_single(X):
    image_descriptors_single = []
    fd, _ = hog(X, orientations=9, pixels_per_cell=(16, 16), cells_per_block=(4, 4),block_norm='L2-Hys', visualize=True)
    image_descriptors_single.append(fd)
    return image_descriptors_single

In [9]:
def read_data():       
    X = []
    pre_img_path_edit=pre_img_path+"rawdata/"
    for img_file in os.listdir(pre_img_path_edit):                       # os.listdir用于返回指定的文件夹包含的文件或文件夹的名字的列表，此处遍历每个文件夹
            path= os.path.join(pre_img_path_edit, img_file)        # 读取图像
            image = np.fromfile(path, dtype=dtype).reshape(shape)
            if image is not None:
                result = preprocessing(image)
                X.append(result)                                    # 将读取到的所有图像的矩阵形式拼接在一起
    return X                                        # 返回的X分别是图像的矩阵表达和图像的标签

In [10]:
face=pd.read_csv(pre_path+"faceLabel.csv")
X=read_data()

In [11]:
print(X[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
face.head()

,number,sex,age,race,emotion,extra,sex_label,age_adult,age_child,age_senior,age_teen,race_asian,race_black,race_hispanic,race_other,race_white,emotion_funny,emotion_serious,emotion_smiling
0,1223,male,child,white,smiling,NaN,1,0,1,0,0,0,0,0,0,1,0,0,1
1,1224,male,child,white,serious,NaN,1,0,1,0,0,0,0,0,0,1,0,1,0
2,1225,male,child,white,smiling,NaN,1,0,1,0,0,0,0,0,0,1,0,0,1
3,1226,male,child,white,smiling,NaN,1,0,1,0,0,0,0,0,0,1,0,0,1
4,1227,male,child,white,serious,NaN,1,0,1,0,0,0,0,0,0,1,0,1,0


In [13]:
num_elements = len(X)
print(f"The dataset has {num_elements} elements.")

The dataset has 3993 elements.


In [14]:
# convert dataframe to tensor
# label_dataFrame=(face["sex_label"])
# label_dataFrame=(face[["sex_label","age_adult","age_child","age_senior","age_teen","race_asian","race_black","race_hispanic","race_other","race_white","emotion_funny","emotion_serious","emotion_smiling"]])
# label_dataFrame=(face[["race_asian","race_black","race_hispanic","race_other","race_white"]])
label_dataFrame=(face[["emotion_funny","emotion_serious","emotion_smiling"]])
# label_dataFrame=(face[["age_adult","age_child","age_senior","age_teen"]])
# label_dataFrame=(face[["sex_label","race_black","race_hispanic","race_other","race_white","emotion_funny","emotion_serious","emotion_smiling"]])

In [15]:
mapping = {'emotion_funny': 0, 'emotion_serious': 1, 'emotion_smiling': 2}
face['emotion_label'] = label_dataFrame.idxmax(axis=1).map(mapping)

In [16]:
print(face['emotion_label'])

0       2
1       1
2       2
3       2
4       1
       ..
3988    2
3989    2
3990    1
3991    2
3992    2
Name: emotion_label, Length: 3993, dtype: int64


In [17]:
Y=face['emotion_label'].values

In [18]:
print(len(X[0]))
print(len(X[0][0]))

128
128


In [19]:
print(Y)

[2 1 2 ... 1 2 2]


In [20]:
def extract_hog_features(X):
    image_descriptors = []
    for i in range(len(X)):
        '''
        参数解释：
        orientations：方向数
        pixels_per_cell：胞元大小
        cells_per_block：块大小
        block_norm：可选块归一化方法L2-Hys(L2范数)
        visualize：可视化
        '''
        fd, _ = hog(X[i], orientations=9, pixels_per_cell=(16, 16), cells_per_block=(4, 4), block_norm='L2-Hys', visualize=True)
        image_descriptors.append(fd)  # 拼接得到所有图像的hog特征
    return image_descriptors        # 返回的是训练部分所有图像的hog特征

In [21]:
X_features = extract_hog_features(X)

In [22]:
print(X_features[0])

[0.         0.         0.         ... 0.02208207 0.06356221 0.02271396]


In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X_features, Y, test_size=0.3, random_state=42)

网格搜索选取SVM参数

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV

gamma_range = np.logspace(-10,1,10)
coef0_range = np.linspace(0,5,10)
C_range = np.linspace(0.01,30,10)
degree_range = np.linspace(1,10,10)
param_grid = dict(gamma = gamma_range
                 ,coef0 = coef0_range
                 ,C = C_range
                 ,degree = degree_range
                 )
cv = KFold(n_splits=5,shuffle=True,random_state=520)
grid = GridSearchCV(SVC(kernel = "poly"),param_grid=param_grid, cv=cv, n_jobs = -1) # n_jobs = -1 调用所有线程
grid.fit(X_train, Y_train)
print("The best parameters are %s with a score of %0.5f" % (grid.best_params_, grid.best_score_))

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV

gamma_range = np.logspace(-10,1,10)
coef0_range = np.linspace(0,5,10)
C_range = np.linspace(0.01,30,20)
param_grid = dict(gamma = gamma_range
                 ,coef0 = coef0_range
                 ,C = C_range
                 )
cv = KFold(n_splits=5,shuffle=True,random_state=520)
grid = GridSearchCV(SVC(kernel = "sigmoid"),param_grid=param_grid, cv=cv, n_jobs = -1) # n_jobs = -1 调用所有线程
grid.fit(X_train, Y_train)
print("The best parameters are %s with a score of %0.5f" % (grid.best_params_, grid.best_score_))

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV

gamma_range = np.logspace(-10,1,10)
C_range = np.linspace(0.01,30,20)
param_grid = dict(gamma = gamma_range
                 ,C = C_range
                 )
cv = KFold(n_splits=5,shuffle=True,random_state=520)
grid = GridSearchCV(SVC(kernel = "rbf"),param_grid=param_grid, cv=cv, n_jobs = -1) # n_jobs = -1 调用所有线程
grid.fit(X_train, Y_train)
print("The best parameters are %s with a score of %0.5f" % (grid.best_params_, grid.best_score_))

In [ ]:
C_range = np.linspace(0.01, 30, 30)
best_acc = -1
best_c = -1
for c in C_range:
    clf = SVC(kernel="linear", C=c)
    clf.fit(X_train, Y_train)
    acc = clf.score(X_test, Y_test)
    if acc > best_acc:
        best_acc = acc
        best_c = c
print("The best c is %0.5f with a score of %0.5f" % (best_c, best_acc))

探究各参数的影响

In [ ]:
score = []
gamma_range = np.logspace(-10, 1, 50) #返回在对数刻度上均匀间隔的数字
for i in gamma_range:
    clf = SVC(kernel="rbf", gamma = i, cache_size=5000).fit(X_train,Y_train)
    score.append(clf.score(X_test, Y_test))
print(max(score), gamma_range[score.index(max(score))])
plt.plot(gamma_range, score)
plt.xlabel('gamma')
plt.ylabel('Accuracy')
plt.savefig('pt1.jpg')
plt.show()

In [ ]:
score = []
C_range = np.linspace(0.01, 30, 30)
for i in C_range:
    clf = SVC(kernel="rbf", C = i, cache_size=5000).fit(X_train,Y_train)
    score.append(clf.score(X_test, Y_test))
print(max(score), C_range[score.index(max(score))])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.savefig('pt2.jpg')
plt.plot(C_range, score)
plt.show()

和其它方式进行对比

In [ ]:
svm = SVC(C = 15.52, kernel='linear')
svm.fit(X_train, Y_train)
Y_predict = svm.predict(X_test)
acc = accuracy_score(Y_test, Y_predict)
print('SVM准确率为: ', acc)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1) # k取1，最近邻准确率较高
knn.fit(X_train,Y_train)
Y_predict = knn.predict(X_test)
acc = accuracy_score(Y_test, Y_predict)
print('KNN准确率为: ', acc)

In [ ]:
tree_D = DecisionTreeClassifier()
tree_D.fit(X_train, Y_train)
Y_predict = tree_D.predict(X_test)
acc = accuracy_score(Y_test, Y_predict)
print('决策树准确率为: ', acc)

In [ ]:
logistic = LogisticRegression()
logistic.fit(X_train, Y_train)
Y_predict = logistic.predict(X_test)
acc = accuracy_score(Y_test, Y_predict)
print('逻辑回归准确率为: ', acc)

In [ ]:
mlt = GaussianNB()
mlt.fit(X_train, Y_train)
Y_predict = mlt.predict(X_test)
acc = accuracy_score(Y_test, Y_predict)
print('朴素贝叶斯准确率为: ', acc)

In [ ]:
Forest = RandomForestClassifier(n_estimators=180,random_state=0)
Forest.fit(X_train, Y_train)
Y_predict = Forest.predict(X_test)
acc = accuracy_score(Y_test, Y_predict)
print('随机森林准确率为: ', acc)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
svc = SVC(C = 20, kernel='poly')
clf = BaggingClassifier(base_estimator=svc, n_estimators=20, max_samples=1.0, max_features=1.0, 
                        bootstrap=True,bootstrap_features=False, n_jobs=-1, random_state=1)
clf.fit(X_train, Y_train)
Y_predict = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_predict)
print('SVM+Bagging准确率为: ', acc)

In [ ]:
from xgboost import XGBClassifier as XGBR
reg = XGBR(n_estimators=200
          ,learning_rate=0.1
          ,booster="gblinear"
          ).fit(X_train,Y_train,eval_metric=['logloss','auc','error'])
Y_predict = reg.predict(X_test)
acc = accuracy_score(Y_test, Y_predict)
print('XGBoost准确率为: ', acc)